In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pydantic import BaseModel, field_validator, Field

## JSON Schema specific limitations
type contraints: no generics, no functions
no custom validation function
no native support for:


## Basic pydantic models

In [ ]:
class StringSimple(BaseModel):
    username: str
    email: str


print(StringSimple.model_json_schema())

In [ ]:
from ipywidgets_jsonschema import Form


def show(model):
    form = Form(model.model_json_schema())
    form.show()

## Nested Model

In [ ]:
from typing import List


class Address(BaseModel):
    street: str
    city: str


class StringNested(BaseModel):
    name: str
    adresses: List[Address]


print(StringNested.model_json_schema())
show(StringNested)

In [ ]:
class JSONContainer:
    def __init__(self, data):
        self._data = data

    def _repr_json_(self):
        return self._data

In [ ]:
JSONContainer(StringNested.model_json_schema())

In [ ]:
class StringAndInt(BaseModel):
    name: str
    age: int


print(StringAndInt.model_json_schema())

In [ ]:
from pydantic import Field


class FloatMultipleOfMinumumContraint(BaseModel):
    price: float = Field(..., multiple_of=0.5, minimum=0.0)


print(FloatMultipleOfMinumumContraint.model_json_schema())
show(FloatMultipleOfMinumumContraint)

In [ ]:
from pydantic import Field


class StringMinLengthMaxLengthContraint(BaseModel):
    username: str = Field(..., min_length=3, max_length=30)
    email: str = Field(..., format="email")


print(StringMinLengthMaxLengthContraint.model_json_schema())
show(StringMinLengthMaxLengthContraint)

In [ ]:
from pydantic import Field


class StringRegex(BaseModel):
    username: str = Field(..., min_length=3, max_length=30, pattern="^[a-zA-Z0-9_]+$")
    email: str = Field(..., format="email")


print(StringRegex.model_json_schema())
show(StringRegex)

In [ ]:
form = Form(StringRegex.model_json_schema())

In [ ]:
form.show()

In [ ]:
form.data

In [ ]:
from typing import List


class Group(BaseModel):
    ids: List[int] = Field(..., min_items=1, max_items=10, set=True)


print(Group.model_json_schema())
show(Group)

In [ ]:
from pydantic import Field


class DictMinProperties(BaseModel):
    name: str
    settings: dict[str, int] = Field(..., min_properties=1)


print(DictMinProperties.model_json_schema())
show(DictMinProperties)

## Custom validator supported in JSON schema

In [ ]:
class CustomValidationFloat(BaseModel):
    discount_type: str
    amount: float
    min_purchase: float

    @field_validator("min_purchase")
    def validate_min_purchase(cls, v, values):
        if v < 50:
            raise ValueError("minimum purchase for discount must be atleast 50")
        return v


print(CustomValidationFloat.model_json_schema())

## Annotations

In [ ]:
class StringWithAnnotations(BaseModel):
    name: str = Field(..., title="Product Name", description="some text")


print(StringWithAnnotations.model_json_schema())
show(StringWithAnnotations)

## Fixed Array Size

In [ ]:
from pydantic import conlist


class FixedArraySize(BaseModel):
    fixed_items: conlist(int, min_length=1, max_length=3)


print(FixedArraySize.model_json_schema())
show(FixedArraySize)

## Custom Validators / Complex Constraints

In [ ]:
class CustomValidationModel(BaseModel):
    username: str
    email: str

    @field_validator("email")
    def validate_call(cls, v):
        if not v.endswith("@example.com"):
            raise ValueError("Wrong domain")
        return v


print(CustomValidationModel.model_json_schema())

# Dynamic fields

In [ ]:
from datetime import datetime


class DynamicFields(BaseModel):
    timestamp: datetime = Field(default_factory=datetime.utcnow)


print(Event.model_json_schema())
show(DynamicFields)

## Aliases

In [ ]:
class IntAlias(BaseModel):
    user_id: int = Field(..., alias="id")


print(IntAlias.model_json_schema())
show(IntAlias)

In [ ]:
class IntField(BaseModel):
    user_id: int = Field(...)


print(IntField.model_json_schema())
show(IntField)

## Private fields

In [ ]:
from pydantic import PrivateAttr


class StringPrivateField(BaseModel):
    _password: str = PrivateAttr()
    _private_password: str

    def __init__(self, password: str):
        self._password = password


print(StringPrivateField.model_json_schema())
show(StringPrivateField)

## Inheritance

In [ ]:
class Animal(BaseModel):
    name: str


class Dog(Animal):
    breed: str


print(Dog.model_json_schema())
show(Dog)

In [ ]:
from typing import List


class Category(BaseModel):
    name: str
    subcategories: List["Category"]


print(Category.model_json_schema())
show(Category)

In [ ]:
JSONContainer(Category.model_json_schema())

## Literals

In [ ]:
from typing import Literal


class LiteralModel(BaseModel):
    category: Literal["clothing", "food"]


print(LiteralModel.model_json_schema())
show(LiteralModel)

In [ ]:
from pydantic import conint


class Item(BaseModel):
    quantity: conint(ge=0)


print(Item.model_json_schema())
show(Item)

## Enums

In [ ]:
from enum import Enum
from typing import List


class Color(Enum):
    RED = "red"
    BLUE = "blue"
    GREEN = "green"


class ListEnum(BaseModel):
    colors: List[Color]


print(ListEnum.model_json_schema())
show(ListEnum)

## Annotated

In [ ]:
from typing import Annotated


class Modelannotated(BaseModel):
    username: Annotated[str, "Must be 12 characters", "Must be alphanumeric"]


print(Modelannotated.model_json_schema())
show(Modelannotated)

## Complex types

In [ ]:
from uuid import UUID


class UUIDModel(BaseModel):
    event_id: UUID


print(UUIDModel.model_json_schema())

## Union

In [ ]:
from typing import Union


class Item(BaseModel):
    price: Union[int, float]


class UnionIntFloat(BaseModel):
    price: int | float


print(UnionIntFloat.model_json_schema())
show(UnionIntFloat)

In [ ]:
from typing import Union, Dict


class DictUnion(BaseModel):
    settings: Dict[str, int]


print(DictUnion.model_json_schema())
show(DictUnion)

In [ ]:
from typing import Union, List, Dict


class DictOrListModel(BaseModel):
    data: Union[str, List[int]]


print(DictOrListModel.model_json_schema())
show(DictOrListModel)

In [ ]:
from typing import Union


class UserA(BaseModel):
    name: str
    age: int


class UserB(BaseModel):
    username: str
    email: str


class UnionModel(BaseModel):
    user: Union[UserA, UserB]


print(UnionModel.model_json_schema())
show(UnionModel)

In [ ]:
from typing import Union
from uuid import UUID

from pydantic import BaseModel


class User(BaseModel):
    id: Union[int, str, UUID]
    name: str


print(User.model_json_schema())
show(User)

user_01 = User(id=123, name="John Doe")
print(user_01)
# > id=123 name='John Doe'
print(user_01.id)
# > 123
user_02 = User(id="1234", name="John Doe")
print(user_02)
# > id='1234' name='John Doe'
print(user_02.id)
# > 1234
user_03_uuid = UUID("cf57432e-809e-4353-adbd-9d5c0d733868")
user_03 = User(id=user_03_uuid, name="John Doe")
print(user_03)
# > id=UUID('cf57432e-809e-4353-adbd-9d5c0d733868') name='John Doe'
print(user_03.id)
# > cf57432e-809e-4353-adbd-9d5c0d733868
print(user_03_uuid.int)
# > 275603287559914445491632874575877060712

In [ ]:
from typing import Literal, Union

from pydantic import BaseModel, Field, ValidationError


class Cat(BaseModel):
    pet_type: Literal["cat"]
    meows: int


class Dog(BaseModel):
    pet_type: Literal["dog"]
    barks: float


class Lizard(BaseModel):
    pet_type: Literal["reptile", "lizard"]
    scales: bool


class AnimalModel(BaseModel):
    pet: Union[Cat, Dog, Lizard] = Field(discriminator="pet_type")
    n: int


print(AnimalModel.model_json_schema())
show(AnimalModel)


print(AnimalModel(pet={"pet_type": "dog", "barks": 3.14}, n=1))
# > pet=Dog(pet_type='dog', barks=3.14) n=1
try:
    AnimalModel(pet={"pet_type": "dog"}, n=1)
except ValidationError as e:
    print(e)
    """
    1 validation error for AnimalModel
    pet.dog.barks
      Field required [type=missing, input_value={'pet_type': 'dog'}, input_type=dict]
    """

In [ ]:
from typing import Literal, Union

from typing_extensions import Annotated

from pydantic import BaseModel, Field, ValidationError


class BlackCat(BaseModel):
    pet_type: Literal["cat"]
    color: Literal["black"]
    black_name: str


class WhiteCat(BaseModel):
    pet_type: Literal["cat"]
    color: Literal["white"]
    white_name: str


Cat = Annotated[Union[BlackCat, WhiteCat], Field(discriminator="color")]


class Dog(BaseModel):
    pet_type: Literal["dog"]
    name: str


Pet = Annotated[Union[Cat, Dog], Field(discriminator="pet_type")]


class Model(BaseModel):
    pet: Pet
    n: int


print(Model.model_json_schema())
show(Model)

m = Model(pet={"pet_type": "cat", "color": "black", "black_name": "felix"}, n=1)
print(m)
# > pet=BlackCat(pet_type='cat', color='black', black_name='felix') n=1
try:
    Model(pet={"pet_type": "cat", "color": "red"}, n="1")
except ValidationError as e:
    print(e)
    """
    1 validation error for Model
    pet.cat
      Input tag 'red' found using 'color' does not match any of the expected tags: 'black', 'white' [type=union_tag_invalid, input_value={'pet_type': 'cat', 'color': 'red'}, input_type=dict]
    """
try:
    Model(pet={"pet_type": "cat", "color": "black"}, n="1")
except ValidationError as e:
    print(e)
    """
    1 validation error for Model
    pet.cat.black.black_name
      Field required [type=missing, input_value={'pet_type': 'cat', 'color': 'black'}, input_type=dict]
    """

## Callables

In [ ]:
from typing import Any, Literal, Union

from typing_extensions import Annotated

from pydantic import BaseModel, Discriminator, Tag


class Pie(BaseModel):
    time_to_cook: int
    num_ingredients: int


class ApplePie(Pie):
    fruit: Literal["apple"] = "apple"


class PumpkinPie(Pie):
    filling: Literal["pumpkin"] = "pumpkin"


def get_discriminator_value(v: Any) -> str:
    if isinstance(v, dict):
        return v.get("fruit", v.get("filling"))
    return getattr(v, "fruit", getattr(v, "filling", None))


class ThanksgivingDinner(BaseModel):
    dessert: Annotated[
        Union[
            Annotated[ApplePie, Tag("apple")],
            Annotated[PumpkinPie, Tag("pumpkin")],
        ],
        Discriminator(get_discriminator_value),
    ]


print(ThanksgivingDinner.model_json_schema())
show(ThanksgivingDinner)


apple_variation = ThanksgivingDinner.model_validate(
    {"dessert": {"fruit": "apple", "time_to_cook": 60, "num_ingredients": 8}}
)
print(repr(apple_variation))
"""
ThanksgivingDinner(dessert=ApplePie(time_to_cook=60, num_ingredients=8, fruit='apple'))
"""

pumpkin_variation = ThanksgivingDinner.model_validate(
    {
        "dessert": {
            "filling": "pumpkin",
            "time_to_cook": 40,
            "num_ingredients": 6,
        }
    }
)
print(repr(pumpkin_variation))
"""
ThanksgivingDinner(dessert=PumpkinPie(time_to_cook=40, num_ingredients=6, filling='pumpkin'))
"""

In [ ]:
from typing import Callable


class CallablesModel(BaseModel):
    process: Callable[[int], int]


print(CallablesModel.model_json_schema())

In [ ]:
class BasicTypes(BaseModel):
    string_field: str
    int_field: int
    float_field: float
    bool_field: bool
    none_field: None

In [ ]:
## Recursion

In [ ]:
class RecursiveModel(BaseModel):
    value: str
    children: List["RecursiveModel"] = []

In [ ]:
from pydantic import SecretStr


class StringValidations(BaseModel):
    min_length: str = Field(min_length=3)
    max_length: str = Field(max_length=10)
    pattern: str = Field(pattern=r"^[a-zA-Z0-9]+$")
    password: SecretStr

In [ ]:
from pydantic import PositiveInt, NegativeInt, conint, PositiveFloat, confloat


class NumericValidations(BaseModel):
    positive_int: PositiveInt
    negative_int: NegativeInt
    range_int: conint(ge=0, le=100)
    positive_float: PositiveFloat
    range_float: confloat(ge=0.0, le=1.0)

In [ ]:
## Enum

In [ ]:
class UserType(str, Enum):
    ADMIN = "admin"
    USER = "user"
    GUEST = "guest"


class EnumModel(BaseModel):
    user_type: UserType
    status: Literal["active", "inactive", "pending"]
    level: int = Field(ge=1, le=5)


show(EnumModel)

In [ ]:
## Nested Model

In [ ]:
from typing import Any


class Address(BaseModel):
    street: str
    city: str
    country: str
    postal_code: str = Field(pattern=r"^\d{5}$")


class NestedModel(BaseModel):
    id: UUID
    name: str
    addresses: List[Address]
    primary_address: Address
    metadata: Dict[str, Any]


show(NestedModel)

## Generics / Template

In [ ]:
from typing import TypeVar, Generic

T = TypeVar("T")


class Generics(BaseModel, Generic[T]):
    data: T
    status: int
    message: str

In [ ]:
JSONContainer(Generics[int].model_json_schema())

## Network model

In [ ]:
from pydantic import BaseModel
from pydantic.networks import IPvAnyAddress


class IpModel(BaseModel):
    ip: IPvAnyAddress


JSONContainer(IpModel.model_json_schema())

## Json

In [ ]:
from pydantic import Json


class DynamicConfig(BaseModel):
    raw_json: Json
    metadata: Dict[str, Any]
    settings_json: Json[Dict[str, Any]]


show(DynamicConfig)

In [ ]:
JSONContainer(DynamicConfig.model_json_schema())

## Optional typing

In [ ]:
from typing import Optional


class OptionalModel(BaseModel):
    field1: Optional[str] = None
    field2: Optional[int] = None
    field3: Optional[List[str]] = None


show(OptionalModel)

In [ ]:
JSONContainer(OptionalModel.model_json_schema())

In [ ]:
import json
import logging
import pydantic
from typing import List, Type
from pathlib import Path


def generate_json_schema_files(
    models: List[Type[BaseModel]], output_dir: str = "schemas"
):
    """
    Generate JSON schema files for a list of pydantic basemodels
    """
    schema_dir = Path(output_dir)
    schema_dir.mkdir(parents=True, exist_ok=True)

    for model in models:
        try:
            model_name = model.__name__.lower()
            schema = model.model_json_schema()
            schema_path = schema_dir / f"{model_name}.json"

            with open(schema_path, "w", encoding="utf8") as f:
                json.dump(schema, f, indent=2, ensure_ascii=False)

            logging.info(f"Generated schema for {model_name} at {schema_path}")

        except Exception as e:
            logging.error(f"Failed to generate schema for {model.__name__}: {str(e)}")


models_to_process = [
    StringSimple,
    StringNested,
    StringAndInt,
    FloatMultipleOfMinumumContraint,
    StringMinLengthMaxLengthContraint,
    StringRegex,
    Group,
    DictMinProperties,
    CustomValidationFloat,
    StringWithAnnotations,
    FixedArraySize,
    CustomValidationModel,
    DynamicFields,
    IntAlias,
    IntField,
    StringPrivateField,
    Dog,
    Category,
    LiteralModel,
    Item,
    ListEnum,
    Modelannotated,
    UUIDModel,
    UnionIntFloat,
    DictUnion,
    BasicTypes,
    RecursiveModel,
    StringValidations,
    NumericValidations,
    EnumModel,
    NestedModel,
    Generics,
    IpModel,
    DynamicConfig,
    OptionalModel,
]
user_generated_models = []
generate_json_schema_files(models_to_process, "schemas")
generate_json_schema_files(user_generated_models, "schemas-user")

In [ ]:
from ipywidgets_jsonschema import Form


# schema = Config.model_json_schema()

form = Form(ListEnum.model_json_schema())
form.show(width="500px")

In [ ]:
form.data

## Unsupported so far
Enums (cant add entries),
Union

## Unsupported by JSON schema
Custom validators, dynamic computed fields